This is a practice in building various recommedation engine based on collaborative filtering using the movielens movie db

-item-item
-user-user
-user-item
-SVD/matrix transformation


In [2]:
import numpy as np
import pandas as pd
import re
import sklearn as skl
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [20]:

# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.user', sep='|', names=u_cols,
 encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.data', sep='\t', names=r_cols,
 encoding='latin-1')

#Reading items file:
i_cols = ['movie_id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.item', sep='|', names=i_cols,
 encoding='latin-1')

In [4]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [5]:
items.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
ratings['rating'].unique()

array([3, 1, 2, 4, 5], dtype=int64)

In [8]:
ratings_base = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\ua.test', sep='\t', names=r_cols, encoding='latin-1')

In [21]:
cols = ['movie_id','movie title']
items2 = items[cols]

In [22]:
rating_items_raw = pd.merge(ratings,items2,how='left',on='movie_id')
rating_items_raw = rating_items_raw.drop('unix_timestamp',axis=1)

In [23]:
average_rating = pd.DataFrame(rating_items_raw.groupby('movie title')['rating'].mean())
average_rating['ratingCount'] = pd.DataFrame(rating_items_raw.groupby('movie title')['rating'].count())
average_rating.sort_values('ratingCount', ascending=False).head(25)

,rating,ratingCount
movie title,,
Star Wars (1977),4.358491,583
Contact (1997),3.803536,509
Fargo (1996),4.155512,508
Return of the Jedi (1983),4.007890,507
Liar Liar (1997),3.156701,485
"English Patient, The (1996)",3.656965,481
Scream (1996),3.441423,478
Toy Story (1995),3.878319,452
Air Force One (1997),3.631090,431


In [24]:
counts1 = rating_items_raw['user_id'].value_counts()
ratings = rating_items_raw[rating_items_raw['user_id'].isin(counts1[counts1 >= 25].index)]
counts = rating_items_raw['rating'].value_counts()
ratings = rating_items_raw[rating_items_raw['rating'].isin(counts[counts >= 10].index)]

In [25]:
ratings_pivot = rating_items_raw.pivot_table(index='user_id', columns='movie title',values='rating', aggfunc='min')
user_id = ratings_pivot.index
movie_title = ratings_pivot.columns
print(ratings_pivot.shape)
ratings_pivot.head()

(943, 1664)


movie title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,Á köldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0,NaN


In [26]:
film = 'Star Wars (1977)'

film_idx = ratings_pivot.columns.get_loc(film)

In [29]:


film_ratings = ratings_pivot[film]
similar_to_film = ratings_pivot.corrwith(film_ratings)
corr_film = pd.DataFrame(similar_to_film, columns=['pearsonR'])
corr_film.dropna(inplace=True)
corr_summary = corr_film.join(average_rating['ratingCount'])
print("recommendation for :", film)
corr_summary[corr_summary['ratingCount']>=300].sort_values('pearsonR', ascending=False).head(10)

C:\Users\suhaib.qazi\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:3154: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\suhaib.qazi\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:3088: RuntimeWarning: divide by zero encountered in double_scalars
  c *= 1. / np.float64(fact)


recommendation for : Star Wars (1977)


,pearsonR,ratingCount
movie title,,
Star Wars (1977),1.000000,583
"Empire Strikes Back, The (1980)",0.747981,367
Return of the Jedi (1983),0.672556,507
Raiders of the Lost Ark (1981),0.536117,420
Indiana Jones and the Last Crusade (1989),0.350107,331
E.T. the Extra-Terrestrial (1982),0.303619,300
Back to the Future (1985),0.274839,350
"Terminator, The (1984)",0.262255,301
"Princess Bride, The (1987)",0.259711,324
